## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два текста: Анну Каренину и Бесов. 

In [82]:
#https://colab.research.google.com/\
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/anna_karenina_part.txt
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/besy_dostoevsky_part.txt
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/avidreaders.ru__mertvye-dushi.txt
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/stoplist_ru.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1350k  100 1350k    0     0  3184k      0 --:--:-- --:--:-- --:--:-- 3184k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1206k  100 1206k    0     0  2949k      0 --:--:-- --:--:-- --:--:-- 2949k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1342k  100 1342k    0     0  1330k      0  0:00:01  0:00:01 --:--:-- 1330k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0    486      0 --:--:-- --:--:-- --:--:--   486


In [0]:
(file1, enc1) = ("anna_karenina_part.txt", "utf-8")
(file2, enc2) = ("besy_dostoevsky_part.txt", "utf-8")
(file3, enc3) = ("avidreaders.ru__mertvye-dushi.txt", "utf-8")


In [0]:
text1 = open(file1, encoding = enc1).read()
text2 = open(file2, encoding = enc2).read()
text3 = open(file3, encoding = enc3).read()
#print(text1[:20])
# print(text2[:20])
# print(text3[:20])

Анна Каренина немного больше.

In [85]:
print(f"Длина {file1} - ", len(text1))
print(f"Длина {file2} - ", len(text2))
print(f"Длина {file3} - ", len(text3))

Длина anna_karenina_part.txt -  769151
Длина besy_dostoevsky_part.txt -  683391
Длина avidreaders.ru__mertvye-dushi.txt -  762191


Напишем простую функцию для нормализации. 

In [86]:

stoplist_ru = set(open("stoplist_ru.txt", mode = 'r', encoding = "utf-8").read().split())
stoplist_ru
#len(stoplist_ru)

{'а',
 'был',
 'была',
 'были',
 'в',
 'его',
 'ее',
 'и',
 'их',
 'не',
 'он',
 'она',
 'они',
 'тот',
 'этот'}

In [0]:
from string import punctuation
import numpy as np

def normalize(text, del_stop = False):
    normalized_text = [word.strip(punctuation) for word in text.lower().split()]
                                                            
    normalized_text = [word for word in normalized_text if word]
    if del_stop:
      normalized_text = [word for word in normalized_text if word not in stoplist_ru]
    return normalized_text


Сравним тексты по словам

In [0]:
stop = False
norm_text1 = normalize(text1, stop)
norm_text2 = normalize(text2, stop)
norm_text3 = normalize(text3, stop)
#del text1, text2, text3

In [89]:
print(f"Длина {file1} - {len(norm_text1)} токенов" )
print(f"Длина {file2} - {len(norm_text2)} токенов" )
print(f"Длина {file3} - {len(norm_text3)} токенов" )

Длина anna_karenina_part.txt - 126487 токенов
Длина besy_dostoevsky_part.txt - 109620 токенов
Длина avidreaders.ru__mertvye-dushi.txt - 119962 токенов


"Мертвые души" короче фрагмента из "Анны Карениной", но уникальных слов там больше!
Бесы короче примерно на 17 тыс. токенов, а уникальных слов всего на 1 тыс. меньше

In [90]:
print(f"Уникальных словоформ в {file1} - {len(set(norm_text1))}")
print(f"Уникальный словоформ в {file2} - {len(set(norm_text2))}")
print(f"Уникальных словоформ в {file3} - {len(set(norm_text3))}")

Уникальных словоформ в anna_karenina_part.txt - 21114
Уникальный словоформ в besy_dostoevsky_part.txt - 21009
Уникальных словоформ в avidreaders.ru__mertvye-dushi.txt - 25903


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [0]:
from collections import Counter

In [0]:
vocab_text1 = Counter(norm_text1)
vocab_text2 = Counter(norm_text2)
vocab_text3 = Counter(norm_text3)


In [93]:
vocab_text1.most_common(10)

[('и', 5851),
 ('–', 5221),
 ('не', 2908),
 ('что', 2620),
 ('в', 2498),
 ('он', 2317),
 ('на', 1624),
 ('она', 1537),
 ('с', 1506),
 ('я', 1392)]

In [94]:
vocab_text2.most_common(10)

[('и', 4551),
 ('—', 3649),
 ('в', 2549),
 ('не', 2425),
 ('что', 1816),
 ('я', 1772),
 ('с', 1342),
 ('на', 1227),
 ('он', 1224),
 ('но', 959)]

In [95]:
vocab_text3.most_common(10)

[('и', 4851),
 ('–', 3788),
 ('в', 2960),
 ('не', 2447),
 ('что', 1982),
 ('на', 1734),
 ('он', 1341),
 ('с', 1283),
 ('как', 1225),
 ('а', 888)]

-------------------------------------------------------------
*Небольшая вставка - что такое Python Counter:

A Counter is a container that keeps track of how many times equivalent values are added. It can be used to implement the same algorithms for which bag or multiset data structures are commonly used in other languages.

Initializing
Counter supports three forms of initialization. Its constructor can be called with a sequence of items, a dictionary containing keys and counts, or using keyword arguments mapping string names to counts.


https://pymotw.com/2/collections/counter.html <br>
https://pythonworld.ru/moduli/modul-collections.html


In [96]:
import collections

print(collections.Counter(['a', 'b', 'c', 'a', 'b', 'b']))
print(collections.Counter({'a':2, 'b':3, 'c':1}))
print(collections.Counter(a=2, b=3, c=1))
#The results of all three forms of initialization are the same.

Counter({'b': 3, 'a': 2, 'c': 1})
Counter({'b': 3, 'a': 2, 'c': 1})
Counter({'b': 3, 'a': 2, 'c': 1})


Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [0]:
def probas_vocab(vocab):
  len_text = len(vocab)
  probas_text = Counter({word : c / len_text for word, c in vocab.items()})
  return probas_text


In [124]:
probas_text1 = probas_vocab(vocab_text1)
print(f"{file1}\n"+"\n".join(str(x) for x in probas_text1.most_common(20)))


anna_karenina_part.txt
('и', 0.2771147106185469)
('–', 0.24727668845315903)
('не', 0.13772852136023492)
('что', 0.12408828265605759)
('в', 0.11831012598276025)
('он', 0.10973761485270436)
('на', 0.07691579047077768)
('она', 0.07279530169555745)
('с', 0.07132708155726059)
('я', 0.06592782040352373)
('как', 0.05688168987401724)
('но', 0.05484512645637965)
('его', 0.05152979066022544)
('это', 0.04750402576489533)
('к', 0.041441697451927634)
('ее', 0.03949985791418016)
('все', 0.035521454958795114)
('было', 0.032821824381926684)
('а', 0.029317040825992232)
('так', 0.02889078336648669)


In [122]:
probas_text2 = Counter({word:c/len(norm_text2) for word, c in vocab_text2.items()})
probas_text2.most_common(20)

[('и', 0.04151614668856048),
 ('—', 0.0332877212187557),
 ('в', 0.023253056011676702),
 ('не', 0.022121875570151432),
 ('что', 0.016566320014595876),
 ('я', 0.016164933406312715),
 ('с', 0.01224229155263638),
 ('на', 0.011193212917350849),
 ('он', 0.01116584564860427),
 ('но', 0.008748403575989784),
 ('вы', 0.008210180623973728),
 ('а', 0.0082010582010582),
 ('как', 0.007370917715745302),
 ('же', 0.006814449917898193),
 ('это', 0.006449553001277139),
 ('его', 0.005956942163838716),
 ('так', 0.0053639846743295016),
 ('к', 0.00531837255975187),
 ('она', 0.0051268016785258165),
 ('всё', 0.004643313264002919)]

In [123]:
probas_text3 = probas_vocab(vocab_text3)
print(f"{file3}\n"+"\n".join(str(x) for x in probas_text3.most_common(20)))

avidreaders.ru__mertvye-dushi.txt
('и', 0.18727560514226152)
('–', 0.146237887503378)
('в', 0.11427247809134077)
('не', 0.09446782225996989)
('что', 0.07651623364089101)
('на', 0.06694205304404895)
('он', 0.051770065243408096)
('с', 0.04953094236188858)
('как', 0.04729181948036907)
('а', 0.03428174342740223)
('я', 0.029340230861290197)
('его', 0.028297880554375942)
('так', 0.028066247152839438)
('все', 0.02667644674362043)
('бы', 0.025402463035169673)
('же', 0.02536385746824692)
('было', 0.024205690460564415)
('но', 0.02397405705902791)
('это', 0.02378102922441416)
('чичиков', 0.023240551287495657)


Эти вероятности уже можно использовать, чтобы ответить на вопрос - кто из авторов сказал бы такую фразу?

In [166]:
phrase = 'Жена узнала, что муж был в связи'

prob = Counter({'tolstoy':0, 'dostoevsky':0, 'gogol':0})
prob_raw = Counter({'tolstoy':1, 'dostoevsky':1, 'gogol':1})

for word in normalize(phrase):
    prob['tolstoy'] += np.log(probas_text1.get(word, 0.00001))
    prob['dostoevsky'] += np.log(probas_text2.get(word, 0.00001))
    prob['gogol'] += np.log(probas_text3.get(word, 0.00001))
    prob_raw['tolstoy'] *= probas_text1.get(word, 0.00001)
    prob_raw['dostoevsky'] *= probas_text2.get(word, 0.00001)
    prob_raw['gogol'] *= probas_text3.get(word, 0.00001)
    print(f"{probas_text1.get(word, 0.00001):.10f}  {probas_text2.get(word, 0.00001):.10f}  {probas_text3.get(word, 0.00001):.10f} {word}")



0.0023680970  0.0000821018  0.0006176891 жена
0.0008998769  0.0000821018  0.0000386056 узнала
0.1240882827  0.0165663200  0.0765162336 что
0.0018471157  0.0000456121  0.0002702390 муж
0.0184711566  0.0029556650  0.0166776049 был
0.1183101260  0.0232530560  0.1142724781 в
0.0008525149  0.0001185915  0.0000386056 связи


-------
небольшая вставка про  https://www.geeksforgeeks.org/numpy-log-python/

In [167]:
print(prob.most_common())
prob_raw.most_common()

[('tolstoy', -34.633123830662726), ('gogol', -44.76306777592244), ('dostoevsky', -51.53599792630299)]


[('tolstoy', 9.099665523991813e-16),
 ('gogol', 3.62782786167172e-20),
 ('dostoevsky', 4.151456400512162e-23)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова незовисымыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [168]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [177]:
sentences_text2 = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(text2)]
sentences_text1 = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(text1)]
sentences_text3 = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(text3)]
print(f"количество предложений в {file1} - {len(sentences_text1)}")
print(f"количество предложений в {file2} - {len(sentences_text2)}")
print(f"количество предложений в {file3} - {len(sentences_text3)}")
sentences_text1 = sentences_text1[:3000]
sentences_text2 = sentences_text2[:3000]
sentences_text2 = sentences_text2[:3000]

количество предложений в anna_karenina_part.txt - 8915
количество предложений в besy_dostoevsky_part.txt - 7634
количество предложений в avidreaders.ru__mertvye-dushi.txt - 7576


In [0]:
def ngrams(sentences):
    unigrams = Counter()
    bigrams = Counter()

    for sentence in sentences:
        unigrams.update(sentence)
        bigrams.update(ngrammer(sentence))
    return (unigrams, bigrams)


In [0]:
(unigrams_text1, bigrams_text1) = ngrams(sentences_text1)
(unigrams_text2, bigrams_text2) = ngrams(sentences_text2)
(unigrams_text3, bigrams_text3) = ngrams(sentences_text3)



In [186]:
len(unigrams_text1)

9321

In [187]:
len(unigrams_text2)

11298

In [188]:
len(unigrams_text3)

25892

In [189]:
bigrams_text1.most_common(10)

[('<start> –', 1060),
 ('– сказал', 165),
 ('<start> она', 137),
 ('<start> и', 117),
 ('степан аркадьич', 115),
 ('<start> он', 114),
 ('– сказала', 98),
 ('– я', 95),
 ('<start> но', 91),
 ('что он', 77)]

In [190]:
bigrams_text2.most_common(10)

[('<start> —', 683),
 ('<start> я', 145),
 ('степан трофимович', 133),
 ('<start> он', 99),
 ('фр <end>', 96),
 ('<start> но', 91),
 ('и не', 90),
 ('варвара петровна', 86),
 ('что он', 72),
 ('— я', 66)]

In [191]:
bigrams_text3.most_common(10)

[('<start> –', 2098),
 ('– сказал', 413),
 ('– да', 194),
 ('<start> в', 174),
 ('<start> но', 159),
 ('и не', 155),
 ('<start> и', 155),
 ('<start> он', 152),
 ('– а', 152),
 ('и в', 150)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [0]:
def prob_sent(phrase, bigrams, unigrams):
  prob = 0
  for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    if word1 in unigrams and ngram in bigrams:
        prob += np.log(bigrams[ngram]/unigrams[word1])
    else:
        prob += -10
  return prob

In [0]:
#phrase = 'Нужно быть действительно великим человеком, чтобы суметь устоять даже против здравого смысла.'
phrase = 'Все смешалось в доме облонских'
probs_sents = Counter(Tolstoy = prob_sent(phrase, bigrams_text1, unigrams_text1), Dostoevsky = prob_sent(phrase, bigrams_text2, unigrams_text2), Gogol = prob_sent(phrase, bigrams_text3, unigrams_text3))

In [200]:
print(prob.most_common())
probs_sents.most_common()

[('tolstoy', -34.633123830662726), ('gogol', -44.76306777592244), ('dostoevsky', -51.53599792630299)]


[('Tolstoy', -18.91589633142466),
 ('Gogol', -49.52445391777101),
 ('Dostoevsky', -50.47792797548535)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [217]:
matrix_text1 = np.zeros((len(unigrams_text1), 
                   len(unigrams_text1)))
id2word_text1 = list(unigrams_text1)
word2id_text1 = {word:i for i, word in enumerate(id2word_text1)}

counter = 10
for ngram in bigrams_text1:
    word1, word2 = ngram.split()
    matrix_text1[word2id_text1[word1]][word2id_text1[word2]] =  (bigrams_text1[ngram]/
                                                                     unigrams_text1[word1])

for i in range(0,len(unigrams_text1)):
  for j in range(0,len(unigrams_text1)):
    if counter > 0 and matrix_text1[i,j] > 0:
      print(matrix_text1[i,j])
      counter -= 1

0.0003333333333333333
0.35333333333333333
0.01
0.0003333333333333333
0.0003333333333333333
0.005
0.006
0.011333333333333334
0.0003333333333333333
0.001
0.5


In [226]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_text2 = np.zeros((len(unigrams_text2), 
                   len(unigrams_text2)))
id2word_text2 = list(unigrams_text2)
word2id_text2 = {word:i for i, word in enumerate(id2word_text2)}


# вероятность расчитываем точно также
for ngram in bigrams_text2:
    word1, word2 = ngram.split()
    matrix_text2[word2id_text2[word1]][word2id_text2[word2]] =  (bigrams_text2[ngram]/
                                                                     unigrams_text2[word1])



[0. 0. 0. ... 0. 0. 0.]


Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [234]:
mtrx = np.zeros((3,3))
mtrx[1] = [0.3, 0.2, 0.5]
print(mtrx.shape)
print(mtrx[1])
count = Counter({1:0, 2:0, 0:0})
for i in range (0,20): 
  count[np.random.choice(mtrx.shape[1], p=mtrx[1])] += 1
  
print(count.most_common())  

(3, 3)
[0.3 0.2 0.5]
[(2, 12), (1, 5), (0, 3)]


In [0]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [221]:
print(generate(matrix_text1, id2word_text1, word2id_text1).replace('<end>', '\n'))

замер 
 надолго с наслаждением полною грудью она встретила вронского с тобой стараемся поскорее наесться чтобы левин сердито сказал он поднял один из маленькой муфты висевшей на вошедшего и она прошла мимо 
 брата моего ребенка и сердце 
 ты зачем я все в дом стало досадно и всегда и ни к ней для меня поняла зачем ты для левина 
 «а может простить то что ли 
 очень мила 
 – вот возьми на середине мазурки с ног его решительным легким шагом вывернутых ног его мысли 
 как хочешь я иначе 
 на его произведет на вальсировавшую анну 



In [0]:
print(generate(matrix_tolstoy, id2word_tolstoy, word2id_tolstoy).replace('<end>', '\n'))

– сказала вдруг как он считал своим шуточным тоном – проговорил он жадно всматривался 
 vii приехав с тою любовью – я не мог простить 
 она беспрестанно как краснеют мальчики – стало быть заряженные пистолеты 
 – сказала она любила этого дня на тело подошел с развратным отцом и хотят вывести из тех влюблений которые все запиской в дверях двинулась было в то в то что и ступай – писала ей несколько лет с облонским стали такие мускулы да – он собирался сделать – сказал степан аркадьич улыбаясь на него все записываю 
 вронский и вронский выехал из моих


## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

In [0]:
import itertools
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
morph = MorphAnalyzer()

def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams
    

Предобработаем почти также, только теперь нам не нужны тэги начала и конца.

In [0]:
sentences_dostoevsky =  [normalize(text) for text in sent_tokenize(dostoevsky)]
sentences_tolstoy =  [normalize(text) for text in sent_tokenize(tolstoy)]


В списке много всяких чисел, однобуквеных слов и стоп-слов. 

Добавим какие-нибудь ограничения к коду выше, чтобы биграммы получались почище.

In [0]:
from nltk.corpus import stopwords

In [0]:
stops = set(stopwords.words('russian') + ['–'])

In [0]:
def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append('_'.join(tokens[i:i+n]))
    return ngrams

In [0]:
word_counter = Counter()

for text in sentences_tolstoy:
    word_counter.update(ngrammer(text, n=2, stops=stops))


In [0]:
word_counter.most_common(15)

[('алексей_александр', 577),
 ('степан_аркадьй', 549),
 ('сергей_иван', 294),
 ('дарья_александр', 209),
 ('весь_это', 171),
 ('сказать_левин', 155),
 ('сказать_степан', 114),
 ('лидий_иван', 104),
 ('сказать_вронский', 88),
 ('сказать_анна', 88),
 ('знать_это', 87),
 ('говорить_это', 86),
 ('агафья_михайло', 76),
 ('графиня_лидий', 74),
 ('сказать_кить', 62)]

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности. 

Такая формула называется PMI.

In [0]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = bigram_count/((word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

Сделаем функцию, которая будет делать счетчики для слов и биграммов.

In [0]:
def collect_stats(texts, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for text in texts:
        unigrams.update(text)
        bigrams.update(ngrammer(text, stops, 2))
    
    return unigrams, bigrams

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.

In [0]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=5):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [0]:
unigrams, bigrams = collect_stats(sentences_tolstoy, stops)

In [0]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [0]:
bigram2score.most_common(15)

[('человек_который', 61.0),
 ('знать_это', 43.5),
 ('первое_время', 32.0),
 ('это_мочь', 30.5),
 ('это_весь', 29.0),
 ('это_дело', 28.5),
 ('это_время', 26.0),
 ('графиня_лидий', 24.666666666666668),
 ('левин_чувствовать', 24.0),
 ('несмотря_весь', 23.0),
 ('дело_который', 22.0),
 ('это_самый', 22.0),
 ('левин_видеть', 22.0),
 ('левин_мочь', 19.0),
 ('левин_понять', 19.0)]

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма.

In [0]:
def scorer(word_count_a, word_count_b, bigram_count, len_vocab, min_count):
    try:
        score = ((bigram_count - min_count) / ((word_count_a + word_count_b)))
    except ZeroDivisionError:
        return 0
    
    return score

In [0]:
bigram2score = score_bigrams(unigrams, bigrams, scorer)

In [0]:
bigram2score.most_common(15)

[('человек_который', 56.0),
 ('знать_это', 41.0),
 ('это_мочь', 28.0),
 ('первое_время', 27.0),
 ('это_весь', 26.5),
 ('это_дело', 26.0),
 ('это_время', 23.5),
 ('графиня_лидий', 23.0),
 ('это_самый', 19.5),
 ('левин_чувствовать', 19.0),
 ('несмотря_весь', 18.0),
 ('дело_который', 17.0),
 ('левин_видеть', 17.0),
 ('это_сказать', 16.0),
 ('друг_друг', 15.333333333333334)]

В статье про Word2Vec для создания нграммов использовалась такая функция:

In [0]:
def scorer_w2v(word_count_a, word_count_b, bigram_count, len_vocab, min_count=10):

    try:
        score = ((bigram_count - min_count) / (word_count_a * word_count_b)) * len_vocab
    except ZeroDivisionError:
        return 0
    
    return score

Посмотрим, отличается ли она от нашей.

In [0]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_w2v)

In [0]:
bigram2score.most_common(15)

[('ребёнок_который', 34925.333333333336),
 ('решить_ехать', 14968.0),
 ('редко_бывать', 14968.0),
 ('решить_это', 9978.666666666666),
 ('решительно_знать', 9978.666666666666),
 ('ребёнок_мочь', 4989.333333333333),
 ('решительно_понимать', 4989.333333333333),
 ('mademoiselle_linon', 2494.6666666666665),
 ('железный_дорога', 2204.589147286822),
 ('женщина_который', 2204.589147286822),
 ('желать_это', 1624.4341085271317),
 ('сергей_иван', 1198.2692520775622),
 ('жена_посланник', 1160.3100775193798),
 ('брат_николай', 1116.0350877192982),
 ('женщина_это', 464.1240310077519)]

Во всех случаях выше мы считали нграммами только слова, которые встречаются друг за другом. Но в нграммы часто можно ещё что-то вставить. Например, "принять участие" может превратиться в "принять самое активное/непосредственное участие". 

Чтобы отловить такие случаи можно считать нграммами слова, которые встречаются внутри какого-то окна. И считать по ним все те же метрики.

Можно ещё посчитать стандартное отклонение расстояния между двумя словами. Если оно маленькое - слова обычно стоят на строгой позиции по отношению друг к другу.

In [0]:
from collections import defaultdict
def get_window_stats(texts, window=8):
    
    bigrams = defaultdict(list)
    
    # проходим окном по текстам 
    # берем первое слово и считаем его целевым
    # проходим по остальным словам и их индексам
    # добавляем в словарь пары (целевое слов, текущее слово)
    # и добавляем индекс текущего в список этой пары
    # так мы получаем (слово_1,слово_2):[1,2,1,1,3,2]
    # порядок в этом случае учитывается - (слово_2, слово_1) - другая запись
    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        # выкидываем биграмы встретившиеся < 5 раз
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [0]:
bigram2std = get_window_stats(sentences_dostoevsky)

In [0]:
bigram2std.most_common()[:-20:-1]

[(('софья', 'матвей'), 0.0),
 (('большой', 'дорога'), 0.0),
 (('ваш', 'превосходительство'), 0.0),
 (('знаешь', 'ли'), 0.0),
 (('общий', 'дело'), 0.0),
 (('арин', 'прохор'), 0.0),
 (('вслед', 'за'), 0.0),
 (('семён', 'яков'), 0.0),
 (('воротиться', 'домой'), 0.0),
 (('из', 'сила'), 0.0),
 (('cher', 'он'), 0.0),
 (('алексей', 'егор'), 0.0),
 (('господин', 'кармазин'), 0.0),
 (('чуть', 'ли'), 0.0),
 (('артемий', 'павло'), 0.0),
 (('замечать', 'что'), 0.0),
 (('какой-то', 'особенный'), 0.0),
 (('с', 'постель'), 0.0),
 (('ради', 'бог'), 0.0)]

Можно применять расширить размер нграмма, а можно последовательно преобразовывать один и тот же текст, на каждом шагу собирая новые биграммы.

Напишием такую функцию.

In [0]:
def bigram_text(text, bigram2score):
    new_text = []
    i = 0
    
    while i < (len(text)-1):
        bigram = '_'.join((text[i], text[i+1]))
        if bigram in bigram2score:
            new_text.append(bigram)
            i += 2
        else:
            new_text.append(text[i])
            i += 1
    else:
        if i == (len(text)-1):
            new_text.append(text[i])
    
    return new_text

In [0]:
unigrams, bigrams = collect_stats(sentences_dostoevsky, stops)

In [0]:
bigram2score = score_bigrams(unigrams, bigrams, scorer)

In [0]:
sentences_dostoevsky_2 = [bigram_text(sent, bigram2score) for sent in sentences_dostoevsky]

In [0]:
unigrams, bigrams = collect_stats(sentences_dostoevsky_2, stops)

In [0]:
trigram2score = score_bigrams(unigrams, bigrams, scorer)

In [0]:
sentences_dostoevsky_3 = [bigram_text(sent, trigram2score) for sent in sentences_dostoevsky_2]

In [0]:
sentences_dostoevsky_3[4]

['домовый', 'ли', 'хоронить_ведьма_ль_замуж', 'выдавать']

По этой ссылке можно прочитать про другие метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

### Все готовое

Писать все это самому конечно не обязательно.

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы. Так как мы сделали выше. 

In [0]:
import gensim

In [0]:
# собираем статистики
ph = gensim.models.Phrases(texts)

In [0]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про ворд2век и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [0]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[texts])
p2 = gensim.models.phrases.Phraser(ph2)

In [0]:
p2[p[texts[0]]][:20]

['многие',
 'интересоваться',
 'зачем',
 'нужный',
 '«яблоку»',
 'молодёжный',
 'фракция',
 'основной_задача',
 '«молодёжный',
 '«яблока»',
 'являться',
 'привлечение',
 'молодая_человек',
 'к',
 'участие',
 'в',
 'выборы',
 'и',
 'деятельность',
 'партия']

Ну и наконец нграммы есть в нлтк. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [0]:
import nltk
from nltk.collocations import *

In [0]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [0]:
finder2 = BigramCollocationFinder.from_documents(texts)

In [0]:
finder3 = TrigramCollocationFinder.from_documents(texts)

In [0]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('один', 'из'),
 ('тот', 'что'),
 ('а', 'также'),
 ('при', 'это'),
 ('2017', 'год'),
 ('не', 'только'),
 ('точка', 'зрение'),
 ('то', 'есть'),
 ('сей', 'пора'),
 ('тот', 'число'),
 ('куб', 'метр'),
 ('владимир', 'путин'),
 ('2016', 'год'),
 ('тот', 'же'),
 ('потому', 'что'),
 ('миллиард', 'доллар'),
 ('о', 'тот'),
 ('прежде', 'всего'),
 ('кроме', 'тот'),
 ('до', 'сей')]

In [0]:
finder3.nbest(trigram_measures.pmi, 20)

[('1947–2001»', 'monterey', 'ca'),
 ('50-летие', 'rolling', 'stones'),
 ('acs', 'nano', 'letters'),
 ('areva', 'edf', 'alstom'),
 ('armored', 'multi-purpose', 'vehicles'),
 ('atr', 'ленур', 'ислям'),
 ('bad', 'can', 'it'),
 ('bourgeois', '«эпатировать', 'буржуа»'),
 ('bundesanstalt', 'fuer', 'geowissenschaften'),
 ('can', 'it', 'be'),
 ('charge', 'ion', 'battery'),
 ('citizens', '1947–2001»', 'monterey'),
 ('commitment', 'competence', 'consensus'),
 ('corriere', 'della', 'sera'),
 ('della', 'sera', 'папа-на-покой'),
 ('diyanet', 'isleri', 'turk-islam'),
 ('dux', 'recording', 'producers'),
 ('edf', 'alstom', 'schneider'),
 ('egf', 'gazprom', 'monitor'),
 ('espanola', 'чть', 'прад')]